## Parte 2

Se ha cometido un delito y la policía necesita su ayuda. Un detective te dio el informe de la
escena del crimen, pero de alguna manera lo perdiste. Recuerdas vagamente que el crimen fue
un asesinato que ocurrió en algún momento del 15 de enero de 2018 y que tuvo lugar en SQL
City. Empiece por recuperar el informe de la escena del crimen correspondiente de la base de
datos del departamento de policía.

In [1]:
import sqlite3
con = sqlite3.connect('sql-murder-mystery.db')

In [5]:
import pandas as pd

In [19]:
# enlistar tablas para conocer los datos
tables='SELECT name FROM sqlite_master WHERE type="table"'
pd.read_sql(tables, con)

,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


In [89]:
# buscar las pistas en los reportes de crimenes
clues='SELECT description FROM crime_scene_report WHERE city="SQL City" AND date="20180115" AND type="murder"'
pd.read_sql(clues, con)

# The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave"

,description
0,Security footage shows that there were 2 witne...


In [107]:
# primer testigo
witness1='SELECT * FROM person WHERE address_street_name="Northwestern Dr" ORDER BY address_number DESC LIMIT 1'
pd.read_sql(witness1, con)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [109]:
# entrevista testigo 1
interview1='SELECT * FROM interview WHERE person_id=14887'
pd.read_sql(interview1, con)
# I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".

,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...


In [140]:
# personas con gold en el gym y el codigo que vio el testigo
clue11='SELECT * FROM get_fit_now_member WHERE membership_status="gold" AND id LIKE "48Z%"'
pd.read_sql(clue11, con)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [159]:
# placa del vehiculo que vio el testigo
clue12='SELECT * FROM drivers_license WHERE plate_number LIKE "%H42W%" AND gender="male"'
pd.read_sql(clue12, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [105]:
# segundo testigo
witness2='SELECT * FROM person WHERE address_street_name="Franklin Ave" AND name LIKE "%Annabel%"'
pd.read_sql(witness2, con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [103]:
# entrevista testigo 2
interview2='SELECT * FROM interview WHERE person_id=16371'
pd.read_sql(interview2, con)
# I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th. 

,person_id,transcript
0,16371,"I saw the murder happen, and I recognized the ..."


In [139]:
# personas que estuvieron en el gym el 9 de Enero
clue2='SELECT * FROM get_fit_now_check_in WHERE check_in_date="20180109" AND membership_id LIKE "48Z%"'
pd.read_sql(witness2, con)

,membership_id,check_in_date,check_in_time,check_out_time
0,X0643,20180109,957,1164
1,UK1F2,20180109,344,518
2,XTE42,20180109,486,1124
3,1AE2H,20180109,461,944
4,6LSTG,20180109,399,515
5,7MWHJ,20180109,273,885
6,GE5Q8,20180109,367,959
7,48Z7A,20180109,1600,1730
8,48Z55,20180109,1530,1700
9,90081,20180109,1600,1700


Uniendo cabos

In [152]:
# personas con gold membership y que estuvieron el 9 de Enero en el gym
clues=f'SELECT name FROM ({clue11}) AS c1 JOIN ({clue2}) AS c2 ON c1.id=c2.membership_id'
pd.read_sql(clues, con)

,name
0,Joe Germuska
1,Jeremy Bowers


In [171]:
# conectando personas con las placas del carro que vio el testigo
license_id=f'SELECT license_id FROM person AS p JOIN ({clues}) AS c ON p.name=c.name'
car_owner=f'SELECT * FROM ({clue12}) AS p JOIN ({license_id}) AS l ON p.id=l.license_id'
pd.read_sql(car_owner, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,license_id
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,423327


In [175]:
# nombre del culpable
guilty=f'SELECT name FROM person JOIN ({car_owner}) AS c ON person.license_id=c.license_id'
pd.read_sql(guilty, con)

,name
0,Jeremy Bowers


In [176]:
culpable='Jeremy Bowers'

Cuando encuentres el culpable, deberás de correr este código para saber si tu respuesta es
correcta:

In [177]:
cursor = con.cursor()
sql = f"INSERT INTO solution VALUES (1, '{culpable}')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [186]:
# entrevista culpable
guilty_id=f'SELECT id FROM person WHERE name="{culpable}"'
interview_guilty='SELECT * FROM interview WHERE person_id="67318"'
pd.read_sql(interview_guilty, con)
# I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.

,person_id,transcript
0,67318,I was hired by a woman with a lot of money. I ...


In [237]:
# datos fisicos de la persona
people_suspect='SELECT id as lic_id FROM drivers_license WHERE hair_color="red" AND height<68 AND height>64 AND car_make="Tesla"  AND car_model="Model S"'
credentials=f'SELECT id, name FROM person JOIN ({people_suspect}) AS s WHERE person.license_id=s.lic_id'
pd.read_sql(credentials, con)

,id,name
0,78881,Red Korb
1,90700,Regina George
2,99716,Miranda Priestly


In [243]:
# concierto al que fue
concert='SELECT COUNT(*) AS att_times, person_id FROM facebook_event_checkin WHERE event_name="SQL Symphony Concert" AND date LIKE "201712%" GROUP BY person_id'
concert3=f'SELECT person_id, att_times FROM ({concert}) WHERE att_times=3'
pd.read_sql(concert3, con)

,person_id,att_times
0,24556,3
1,99716,3


In [239]:
# nombre del culpable real
real_guilty=f'SELECT name FROM ({credentials}) as c JOIN ({concert3}) AS c3 ON c.id=c3.person_id'
pd.read_sql(real_guilty, con)

,name
0,Miranda Priestly


In [240]:
culpable_real='Miranda Priestly'

In [241]:
cursor = con.cursor()
sql = f"INSERT INTO solution VALUES (1, '{culpable_real}')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'